In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Average
from keras.utils import np_utils
from keras import backend as K
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

batch_size = 128
nb_classes = 2
nb_epoch = 10

# input image dimensions
img_rows, img_cols = 20, 20
# number of convolutional filters to use
nb_filters = 64
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)
input_shape=(20, 20,1)

Using TensorFlow backend.
/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/h

In [2]:

vetor_atendente_final=pd.read_excel('Atendentes_Vetores.xlsx',header=0,index=False)
vetor_atendente_final.shape

(2771, 401)

In [3]:
vetor_cliente_final=pd.read_excel('Clientes_Vetores.xlsx',header=0,index=False)
vetor_atendente_final.iloc[:,0:-1].shape

(2771, 400)

In [4]:
target=np.array(pd.get_dummies(vetor_atendente_final.iloc[:,-1]))
target_=target
target

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [63]:
def norm2(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

In [64]:
tabela_dados=pd.read_excel('TabelaDados.xlsx',header=0,index=False)
tabela_dados2=tabela_dados.iloc[np.concatenate([np.where(target.T[1]==1)[0][0:1680],np.where(target.T[1]==0)[0][-1091:]],axis=0),:].fillna(0.9)
tabela_dados2['Satisfação cliente']=tabela_dados2['Satisfação cliente'].astype('category').cat.codes
tabela_dados2['Satisfação atendente']=tabela_dados2['Satisfação atendente'].astype('category').cat.codes
tabela_dados3=tabela_dados2[['Silencio','Tempo de Ligação','Atend_Lex','Cliente_Lex','Satisfação cliente',
                                     'Satisfação atendente']]
tabela_dados3=norm2(tabela_dados3)

In [65]:
tabela_dados3['Silencio']

0       0.700646
1       0.701806
2       0.062971
3       0.533697
4       0.068154
          ...   
2766    0.704984
2767    0.067725
2768    0.697560
2769    0.678057
2770    0.679891
Name: Silencio, Length: 2771, dtype: float64

In [66]:
bins = [0, 15, 25, 35, 50, 75,85, 100]
labels = [1,2,3,4,5,6,7]
for i in tabela_dados3.columns[0:4]:
    tabela_dados3[i] = pd.cut(tabela_dados3[i], bins=bins,labels=labels)
tabela_dados3['Silencio']

0       1
1       1
2       1
3       1
4       1
       ..
2766    1
2767    1
2768    1
2769    1
2770    1
Name: Silencio, Length: 2771, dtype: category
Categories (7, int64): [1 < 2 < 3 < 4 < 5 < 6 < 7]

In [67]:
#tabela_dados3=np.array(tabela_dados3.iloc[:,[2,3]])

variaveis=tabela_dados3.shape[1]
concat=np.array(tabela_dados3)
concat=concat.reshape(-1,variaveis,1)
concat;

In [68]:
def norm(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

x_train0=np.array(norm(np.array(vetor_atendente_final.iloc[:,0:-1]))).reshape(-1,20,20,1)
x_train1=np.array(norm(np.array(vetor_cliente_final.iloc[:,0:-1]))).reshape(-1,20,20,1)
x_train1.shape

(2771, 20, 20, 1)

In [69]:
from sklearn.model_selection import train_test_split

X, y = x_train0, target
X1, y1 = x_train1, target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.1, random_state=42)
concat_train,concat_test, y_train2, y_test2=train_test_split(concat, y1, test_size=0.1, random_state=42)

In [70]:
from keras.layers import Input, Dense, Lambda, Concatenate
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.datasets import mnist
from keras.layers.core import Reshape
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, UpSampling2D,Convolution1D,MaxPooling1D
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
import os
from keras.optimizers import SGD
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


with session.as_default():
     with session.graph.as_default():
        
        
        

        input_shape2=(20,20,1)
        input_img = Input(batch_shape=(None, 20,20,1))
        input_img1 = Input(batch_shape=(None, 20,20,1))
        input_img2 = Input(batch_shape=(None, variaveis,1))


        denoise_left=Convolution2D(20, 3,3,
                                border_mode='same',
                                input_shape=input_shape)(input_img)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=MaxPooling2D(pool_size=(2,2))(denoise_left)
        denoise_left=Convolution2D(20, 3, 3,
                                    init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=UpSampling2D(size=(2, 2))(denoise_left)
        denoise_left=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=UpSampling2D(size=(2, 2))(denoise_left)
        denoise_left=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=Convolution2D(1, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('sigmoid')(denoise_left)
        denoise_left=Dense(20)(denoise_left)
        denoise_left=Reshape((-1,20,1))(denoise_left)
        denoise_left=Flatten()(denoise_left)

        denoise_left1=Convolution2D(20, 3,3,
                                border_mode='same',
                                input_shape=input_shape)(input_img1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=MaxPooling2D(pool_size=(2,2))(denoise_left1)
        denoise_left1=Convolution2D(20, 3, 3,
                                    init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=UpSampling2D(size=(2, 2))(denoise_left1)
        denoise_left1=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=UpSampling2D(size=(2, 2))(denoise_left1)
        denoise_left1=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=Convolution2D(1, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('sigmoid')(denoise_left1)
        denoise_left1=Dense(20)(denoise_left1)
        denoise_left1=Reshape((-1,20,1))(denoise_left1)
        denoise_left1=Flatten()(denoise_left1)

        denoise_concat=Dense(20)(input_img2)
        denoise_concat=Activation('relu')(denoise_concat)
        denoise_concat=Dense(20)(denoise_concat)
        denoise_concat=Activation('sigmoid')(denoise_concat)
        denoise_concat=Dense(variaveis)(denoise_concat)
        denoise_concat=Reshape((-1,variaveis,1))(denoise_concat)
        denoise_concat=Flatten()(denoise_concat)

        squeeze0=Concatenate()([denoise_left,denoise_left1,denoise_concat])
        squeeze0=Dropout(0.1)(squeeze0)
        squeeze0=Dense(20)(squeeze0)
        squeeze0=Activation('relu')(squeeze0)
        squeeze0=Dense(2)(squeeze0)
        squeeze0=Activation('sigmoid')(squeeze0)

        model = Model(inputs = [input_img,input_img1,input_img2], outputs = squeeze0)

        model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01,momentum=0.6, decay=5e-5, nesterov=False),metrics = ['accuracy'])
        save = ModelCheckpoint('pesos_keras.hdf5', save_best_only=True, monitor='val_loss', mode='min')

        init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        session.run(init)
        model.fit([X_train, X_train1,concat_train],y_train,
                nb_epoch=600,
                batch_size=2771,verbose=1,validation_split=0.1,callbacks=[save]
                 )
        preds=model.predict([X_test,X_test1,concat_test])

/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), input_shape=(20, 20, 1..., padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:46: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), kernel_initializer="glorot_uniform", padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), kernel_initializer="glorot_uniform", padding="same")`
/usr/local/lib/python3.5/

Train on 2243 samples, validate on 250 samples
Epoch 1/600
2243/2243 [==============================] - 2s 978us/step - loss: 0.7032 - acc: 0.3977 - val_loss: 0.6860 - val_acc: 0.5800
Epoch 2/600
2243/2243 [==============================] - 0s 82us/step - loss: 0.6826 - acc: 0.6068 - val_loss: 0.6801 - val_acc: 0.5800
Epoch 3/600
2243/2243 [==============================] - 0s 79us/step - loss: 0.6726 - acc: 0.6068 - val_loss: 0.6831 - val_acc: 0.5800
Epoch 4/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6709 - acc: 0.6068 - val_loss: 0.6864 - val_acc: 0.5800
Epoch 5/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6714 - acc: 0.6068 - val_loss: 0.6850 - val_acc: 0.5800
Epoch 6/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6714 - acc: 0.6068 - val_loss: 0.6832 - val_acc: 0.5800
Epoch 7/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6705 - acc: 0.6068 - val_loss: 0.6818 - val_acc: 0.5800


Epoch 61/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6574 - acc: 0.6068 - val_loss: 0.6732 - val_acc: 0.5800
Epoch 62/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.6569 - acc: 0.6068 - val_loss: 0.6732 - val_acc: 0.5800
Epoch 63/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6570 - acc: 0.6068 - val_loss: 0.6731 - val_acc: 0.5800
Epoch 64/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.6552 - acc: 0.6068 - val_loss: 0.6730 - val_acc: 0.5800
Epoch 65/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.6543 - acc: 0.6068 - val_loss: 0.6717 - val_acc: 0.5800
Epoch 66/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.6545 - acc: 0.6068 - val_loss: 0.6725 - val_acc: 0.5800
Epoch 67/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6524 - acc: 0.6068 - val_loss: 0.6710 - val_acc: 0.5800
Epoch 68/600
2243/2243 [=================

Epoch 121/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.6398 - acc: 0.6072 - val_loss: 0.6591 - val_acc: 0.5800
Epoch 122/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.6385 - acc: 0.6068 - val_loss: 0.6579 - val_acc: 0.5800
Epoch 123/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6371 - acc: 0.6068 - val_loss: 0.6568 - val_acc: 0.5800
Epoch 124/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6352 - acc: 0.6068 - val_loss: 0.6556 - val_acc: 0.5800
Epoch 125/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6349 - acc: 0.6068 - val_loss: 0.6544 - val_acc: 0.5800
Epoch 126/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.6316 - acc: 0.6072 - val_loss: 0.6531 - val_acc: 0.5800
Epoch 127/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.6297 - acc: 0.6077 - val_loss: 0.6517 - val_acc: 0.5840
Epoch 128/600
2243/2243 [=========

Epoch 181/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.5403 - acc: 0.7263 - val_loss: 0.7089 - val_acc: 0.6160
Epoch 182/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.5393 - acc: 0.7272 - val_loss: 0.6882 - val_acc: 0.6160
Epoch 183/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.5383 - acc: 0.7289 - val_loss: 0.7336 - val_acc: 0.6080
Epoch 184/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.5389 - acc: 0.7209 - val_loss: 0.6810 - val_acc: 0.6160
Epoch 185/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.5375 - acc: 0.7325 - val_loss: 0.7873 - val_acc: 0.5920
Epoch 186/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.5409 - acc: 0.7200 - val_loss: 0.6446 - val_acc: 0.6360
Epoch 187/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.5474 - acc: 0.7240 - val_loss: 0.9307 - val_acc: 0.5840
Epoch 188/600
2243/2243 [=========

...

Epoch 301/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.4431 - acc: 0.7949 - val_loss: 0.8015 - val_acc: 0.6120
Epoch 302/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.4624 - acc: 0.7856 - val_loss: 1.4920 - val_acc: 0.5840
Epoch 303/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.5546 - acc: 0.7098 - val_loss: 0.5761 - val_acc: 0.7160
Epoch 304/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.7062 - acc: 0.6108 - val_loss: 1.9299 - val_acc: 0.5800
Epoch 305/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.9189 - acc: 0.6206 - val_loss: 0.7044 - val_acc: 0.6160
Epoch 306/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.4575 - acc: 0.7976 - val_loss: 0.5908 - val_acc: 0.6560
Epoch 307/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.5220 - acc: 0.7423 - val_loss: 1.0792 - val_acc: 0.5840
Epoch 308/600
2243/2243 [=========

Epoch 361/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.3723 - acc: 0.8489 - val_loss: 1.2833 - val_acc: 0.5840
Epoch 362/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.3688 - acc: 0.8556 - val_loss: 1.2242 - val_acc: 0.5840
Epoch 363/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.3670 - acc: 0.8564 - val_loss: 1.3415 - val_acc: 0.5840
Epoch 364/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.3663 - acc: 0.8538 - val_loss: 1.2220 - val_acc: 0.5840
Epoch 365/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.3636 - acc: 0.8636 - val_loss: 1.3786 - val_acc: 0.5840
Epoch 366/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.3635 - acc: 0.8578 - val_loss: 1.1882 - val_acc: 0.5840
Epoch 367/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.3621 - acc: 0.8605 - val_loss: 1.4431 - val_acc: 0.5840
Epoch 368/600
2243/2243 [=========

Epoch 421/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.2833 - acc: 0.9131 - val_loss: 1.6071 - val_acc: 0.5840
Epoch 422/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.2811 - acc: 0.9068 - val_loss: 1.4206 - val_acc: 0.5880
Epoch 423/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.2792 - acc: 0.9166 - val_loss: 1.6542 - val_acc: 0.5840
Epoch 424/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.2803 - acc: 0.9019 - val_loss: 1.3802 - val_acc: 0.5920
Epoch 425/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.2818 - acc: 0.9131 - val_loss: 1.7606 - val_acc: 0.5840
Epoch 426/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.2910 - acc: 0.8819 - val_loss: 1.2569 - val_acc: 0.5960
Epoch 427/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.3099 - acc: 0.8814 - val_loss: 1.9935 - val_acc: 0.5800
Epoch 428/600
2243/2243 [=========

Epoch 481/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.2144 - acc: 0.9229 - val_loss: 0.9513 - val_acc: 0.6280
Epoch 482/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.2484 - acc: 0.9019 - val_loss: 1.6259 - val_acc: 0.6040
Epoch 483/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.2204 - acc: 0.9140 - val_loss: 1.3756 - val_acc: 0.5960
Epoch 484/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.1887 - acc: 0.9514 - val_loss: 1.2726 - val_acc: 0.5960
Epoch 485/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.1921 - acc: 0.9545 - val_loss: 1.5947 - val_acc: 0.6040
Epoch 486/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.1914 - acc: 0.9407 - val_loss: 1.3003 - val_acc: 0.5960
Epoch 487/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.1820 - acc: 0.9621 - val_loss: 1.4110 - val_acc: 0.5960
Epoch 488/600
2243/2243 [=========

Epoch 541/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.1071 - acc: 0.9871 - val_loss: 1.0628 - val_acc: 0.6280
Epoch 542/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.1049 - acc: 0.9902 - val_loss: 1.1082 - val_acc: 0.6320
Epoch 543/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.1033 - acc: 0.9889 - val_loss: 1.1080 - val_acc: 0.6360
Epoch 544/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.1016 - acc: 0.9902 - val_loss: 1.1377 - val_acc: 0.6280
Epoch 545/600
2243/2243 [==============================] - 0s 75us/step - loss: 0.0998 - acc: 0.9889 - val_loss: 1.1546 - val_acc: 0.6240
Epoch 546/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0978 - acc: 0.9893 - val_loss: 1.1390 - val_acc: 0.6360
Epoch 547/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0961 - acc: 0.9906 - val_loss: 1.1790 - val_acc: 0.6240
Epoch 548/600
2243/2243 [=========

In [71]:
from keras.utils import plot_model 
plot_model(model, to_file='model_gif.png')

In [72]:
model.load_weights('pesos_keras.hdf5')
loss,acc = model.evaluate([X_test, X_test1,concat_test],y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

278/278 [==============================] - 0s 141us/step
Restored model, accuracy: 74.82%


In [79]:
preds00=model.predict([X_test, X_test1,concat_test])
preds00

array([[0.04653198, 0.18678865],
       [0.09754336, 0.14846945],
       [0.04931238, 0.20231482],
       [0.08633527, 0.14121622],
       [0.04283047, 0.20828727],
       [0.29164916, 0.09566882],
       [0.09295866, 0.18208301],
       [0.16304272, 0.13132164],
       [0.31526503, 0.08701658],
       [0.10036233, 0.14645162],
       [0.36909407, 0.05972186],
       [0.05830103, 0.19359303],
       [0.3384604 , 0.07213241],
       [0.08737695, 0.14131469],
       [0.31980813, 0.07022375],
       [0.0415799 , 0.20759684],
       [0.07122862, 0.16217765],
       [0.11568955, 0.14524499],
       [0.15295327, 0.14126039],
       [0.10711941, 0.13682875],
       [0.21072575, 0.1132088 ],
       [0.06847632, 0.14938349],
       [0.04875109, 0.21363845],
       [0.06011391, 0.21905154],
       [0.07228336, 0.15723881],
       [0.0570713 , 0.22016951],
       [0.04046392, 0.21324041],
       [0.14052996, 0.13389552],
       [0.1069873 , 0.15752873],
       [0.09244233, 0.15401208],
       [0.

In [80]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,f1_score,recall_score


In [81]:
confusion_matrix(y_test.T[1],np.argmax(preds00,axis=1))
pred0=np.argmax(preds00,axis=1)
Y=y_test.T[1]
print('\n')
print('Accuracy:',accuracy_score(np.array(Y),pred0))
print('Precision (FP):',precision_score(np.array(Y),pred0,average='binary'))
print('Recall (FN):',recall_score(np.array(Y),pred0,average='binary'))
print('f1:',f1_score(np.array(Y),pred0,average='binary'))
print(confusion_matrix(np.array(Y),pred0),'\n')



Accuracy: 0.7482014388489209
Precision (FP): 0.7524271844660194
Recall (FN): 0.8908045977011494
f1: 0.8157894736842105
[[ 53  51]
 [ 19 155]] 

